In [1]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, merge, Input, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import model_from_json
from keras import backend as K
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
import random
import os
import cv2

from utils import Smooth, BatchGenerator

import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
df = pd.read_csv('driving_log.csv', skipinitialspace=True)
df.columns = ['center', 'left', 'right', 'dir', 'acc', 'nacc', 'speed']
df = df.drop((df[df['acc'] != 1]).index)
df = df.drop((df[df['speed'] < 30.1]).index)
print df['speed'].mean(), df['dir'].mean(), df['dir'].max(), df['dir'].min()
df = df.reset_index(drop=True)
df['dir'] = df['dir'] * 100
df.head()

30.1862628077 -0.00445289741769 1.0 -1.0


,center,left,right,dir,acc,nacc,speed
0,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,-46.68390,1.0,0.0,30.14583
1,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,-82.77869,1.0,0.0,30.10305
2,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,-51.98100,1.0,0.0,30.18893
3,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,-21.23459,1.0,0.0,30.19710
4,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,/Users/ypw/Desktop/a/IMG/left_2017_01_05_10_33...,/Users/ypw/Desktop/a/IMG/right_2017_01_05_10_3...,0.00000,1.0,0.0,30.18751


In [3]:
df = df.drop(['left', 'right', 'acc', 'nacc', 'speed'], axis=1)
df.columns = ['filename', 'dir']
df.head()

,filename,dir
0,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,-46.68390
1,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,-82.77869
2,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,-51.98100
3,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,-21.23459
4,/Users/ypw/Desktop/a/IMG/center_2017_01_05_10_...,0.00000


In [4]:
batch = 64
image_shape = (128, 128)

df_train, df_val = train_test_split(df, test_size=0.05, random_state=42)
train = BatchGenerator(df_train, batch, image_shape)
val = BatchGenerator(df_val, batch, image_shape)

print train.num, val.num

21417 1128


In [5]:
from keras.applications import VGG16

img_input = Input(shape=(image_shape[1], image_shape[0], 3))
base_model = VGG16(weights='imagenet', input_tensor=img_input, include_top=False)

In [6]:
x = base_model.output

x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dense(1)(x)

model = Model(input=base_model.input, output=x)

model.compile(loss='mse', optimizer='adadelta', metrics=['accuracy'])

In [7]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 128, 128, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 128, 128, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 64, 64)    0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [8]:
zip(range(len(model.layers)), model.layers)

[(0, <keras.engine.topology.InputLayer at 0x1115de750>),
 (1, <keras.layers.convolutional.Convolution2D at 0x15c308ed0>),
 (2, <keras.layers.convolutional.Convolution2D at 0x15c327090>),
 (3, <keras.layers.pooling.MaxPooling2D at 0x160db4090>),
 (4, <keras.layers.convolutional.Convolution2D at 0x160de5e90>),
 (5, <keras.layers.convolutional.Convolution2D at 0x160eac310>),
 (6, <keras.layers.pooling.MaxPooling2D at 0x160ee4f50>),
 (7, <keras.layers.convolutional.Convolution2D at 0x160eacad0>),
 (8, <keras.layers.convolutional.Convolution2D at 0x160ef2ad0>),
 (9, <keras.layers.convolutional.Convolution2D at 0x1610fbf50>),
 (10, <keras.layers.pooling.MaxPooling2D at 0x161109450>),
 (11, <keras.layers.convolutional.Convolution2D at 0x161144a10>),
 (12, <keras.layers.convolutional.Convolution2D at 0x161144610>),
 (13, <keras.layers.convolutional.Convolution2D at 0x1611911d0>),
 (14, <keras.layers.pooling.MaxPooling2D at 0x1611cc850>),
 (15, <keras.layers.convolutional.Convolution2D at 0x161

In [9]:
val_num = len(df_val.index)
X_valid = np.ndarray((val_num, image_shape[1], image_shape[0], 3), dtype=np.float32)
y_valid = np.ndarray((val_num, 1), dtype=np.float32)
for i, index in enumerate(df_val.index):
    img = cv2.imread(df_val.loc[index]['filename'])
    img = cv2.resize(img, image_shape)
    X_valid[i] = img
    y_valid[i] = df_val.loc[index]['dir']

def validation():
    loss, acc = model.evaluate(X_valid, y_valid, verbose=0)
    return acc, loss

validation()

(0.023049645390070921, 438.7745818009613)

In [10]:
def train_one_batch(generator):
    acc = Smooth()
    loss = Smooth()
    batch_num = generator.num/batch
    pbar = tqdm(xrange(batch_num))
    for i in pbar:
        X, y = generator.next()
        _loss, _acc = model.train_on_batch(X, y)
        acc += _acc
        loss += _loss
        
        if i != batch_num-1:
            pbar.set_description("train loss: %.4f" % loss)
        else:
            pbar.set_description("train loss: %.4f\t valid loss: %.4f\t" % (loss, validation()[1]))

def transfer_learning(top_num=-4, epoch=10, optimizer='adadelta'):
    for layer in model.layers[:top_num]:
        layer.trainable = False

    for layer in model.layers[top_num:]:
        layer.trainable = True
    
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    
    for i in range(epoch):
        train_one_batch(train)

In [11]:
transfer_learning(18, epoch=6)

train loss: 102.9973	 valid loss: 164.0933	: 100%|██████████| 334/334 [01:23<00:00,  1.04s/it]


In [12]:
transfer_learning(14, epoch=4)

train loss: 110.5120	 valid loss: 159.0068	: 100%|██████████| 334/334 [01:36<00:00,  1.09s/it]


In [13]:
X_val, y_val = val.next()
y_pred = model.predict(X_val)
delta = y_val - y_pred

zip(y_val.tolist(), y_pred.tolist())

[([0.0], [-0.682125985622406]),
 ([26.04660987854004], [2.0592830181121826]),
 ([0.0], [1.5431612730026245]),
 ([25.499990463256836], [1.3266446590423584]),
 ([-7.867490768432617], [-1.7862656116485596]),
 ([-15.260109901428223], [-5.352181434631348]),
 ([13.276789665222168], [5.263405799865723]),
 ([11.388890266418457], [28.26003646850586]),
 ([0.0], [0.2926846146583557]),
 ([0.0], [3.567487955093384]),
 ([0.0], [-4.981549263000488]),
 ([55.76189041137695], [29.68886375427246]),
 ([0.0], [-1.2190297842025757]),
 ([7.254605770111084], [11.70290470123291]),
 ([-21.605690002441406], [-22.61895751953125]),
 ([0.0], [0.06620453298091888]),
 ([0.0], [-3.1688709259033203]),
 ([0.0], [21.272125244140625]),
 ([10.624799728393555], [3.084975481033325]),
 ([40.6875], [-11.323773384094238]),
 ([-16.798189163208008], [0.5051746964454651]),
 ([-6.286794185638428], [-26.85157012939453]),
 ([-5.072397232055664], [-24.06559944152832]),
 ([0.0], [3.161877393722534]),
 ([-24.44399070739746], [-25.362413

In [14]:
model.save('model.h5', overwrite=True)

In [15]:
from keras.applications import VGG16

img_input = Input(shape=(image_shape[1], image_shape[0], 3))
base_model = VGG16(weights='imagenet', input_tensor=img_input, include_top=False)

x = base_model.output

x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dense(1)(x)

model = Model(input=base_model.input, output=x)

model.compile(loss='mse', optimizer='adadelta', metrics=['accuracy'])

with open('model.json', 'w') as f:
    f.write(model.to_json())